In [17]:
import warnings
warnings.filterwarnings('ignore')

import swifter
import importlib.util
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.compose import make_column_selector as selector, ColumnTransformer, make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics



import pandas as pd
import numpy as np
from scipy.stats import skew

In [2]:
parse_times = ["MKOPEN", "MKCLOSE", "MKEMHOPEN", "MKEMHCLOSE",
               "MKOPENYEST", "MKCLOSEYEST", "MKOPENTOM",
               "MKCLOSETOM","EPOPEN", "EPCLOSE", "EPEMHOPEN",
               "EPEMHCLOSE", "EPOPENYEST", "EPCLOSEYEST",
               "EPOPENTOM", "EPCLOSETOM", "HSOPEN", "HSCLOSE",
               "HSEMHOPEN", "HSEMHCLOSE", "HSOPENYEST", "HSCLOSEYEST",
               "HSOPENTOM", "HSCLOSETOM", "AKOPEN", "AKCLOSE",
               "AKEMHOPEN", "AKOPENYEST", "AKCLOSEYEST","AKEMHCLOSE",
               "AKOPENTOM", "AKCLOSETOM", "MKPRDDT1", "MKPRDDT2",
               "MKPRDNT1", "MKPRDNT2", "MKFIRET1", "MKFIRET2",
               "EPFIRET1", "EPFIRET2", "HSPRDDT1", "HSFIRET1",
               "HSFIRET2", "HSSHWNT1", "HSSHWNT2", "AKPRDDT1",
               "AKPRDDT2", "AKSHWNT1", "AKSHWNT2"]

In [3]:
# specify the module that needs to be
# imported relative to the path of the
# module
spec = importlib.util.spec_from_file_location("loadTrainTestPostedWaitTimes", "../src/data/loadTrainTestData.py")

# creates a new module based on spec
loadTrainPosted = importlib.util.module_from_spec(spec)

# executes the module in its own namespace
# when a module is imported or reloaded.
spec.loader.exec_module(loadTrainPosted)

X_train, X_test, y_train, y_test = loadTrainPosted.loadTrainTestPostedWaitTimes()

In [4]:
X_train["MONTHOFYEAR"] = X_train["date"].dt.month.astype("Int8")
X_train["YEAR"] = X_train["date"].dt.year.astype("Int16")
X_train["DAYOFYEAR"] = X_train["date"].dt.dayofyear.astype("Int16")
X_train["HOUROFDAY"] = X_train["datetime"].dt.hour.astype("Int8")

X_test["MONTHOFYEAR"] = X_test["date"].dt.month.astype("Int8")
X_test["YEAR"] = X_test["date"].dt.year.astype("Int16")
X_test["DAYOFYEAR"] = X_test["date"].dt.dayofyear.astype("Int16")
X_test["HOUROFDAY"] = X_test["datetime"].dt.hour.astype("Int8")

In [5]:
train = pd.concat([X_train, y_train], axis=1).sort_values(['datetime'])
test = pd.concat([X_test, y_test], axis=1).sort_values(['datetime'])

X_train_impute = train.drop(columns=["POSTED_WAIT"])
y_train = train["POSTED_WAIT"]

X_test_impute = test.drop(columns=["POSTED_WAIT"])
y_test = test["POSTED_WAIT"]

In [12]:
X_train_clean = X_train_impute.drop(columns=['date', 'datetime', 'Unnamed: 0'])
X_test_clean = X_test_impute.drop(columns=['date', 'datetime', 'Unnamed: 0'])

# Column Transformer

In [13]:
allCols = list(X_train_clean.columns)

def imputeTransform(x):
    for col in x:
        print(col)
        if col in parse_times:
            x[col] = x[col].fillna("99")
            x[col] = x[col].apply(lambda h: h[:2] if h[0]!=0 else h[:1]).astype(int).astype("Int8")
            
        x[col] = x[col].fillna(method ='bfill')
        x[col] = x[col].fillna(x[col].median())
    
            
        if (x[col].dtype != "bool") and (abs(skew(list(x[col]))) > 0.8):
            x[f"log_{col}"] = x[col].swifter.apply(lambda k: np.log(k+20)) # +20 linear scale on all values to ensure no resulting -inf vals
            x.drop(columns=[col], inplace=True)
    return x

preprocessor = make_column_transformer(
      (VarianceThreshold(threshold=0.001), selector(dtype_include="bool")),
      (RobustScaler(), selector(dtype_include=np.number)), remainder='passthrough')


clf = Pipeline(
    steps=[("imputerAndLogTransformer", FunctionTransformer(imputeTransform)),
           ("preprocessor", preprocessor), 
           ("regressor", RandomForestRegressor(n_estimators=50, max_depth=50, n_jobs=-1, random_state=0))]
)

In [14]:
clf.fit(X_train_clean, y_train)

Ride_type_thrill
Ride_type_spinning
Ride_type_slow
Ride_type_small_drops
Ride_type_big_drops
Ride_type_dark
Fast_pass
Classic
Age_interest_preschoolers
Age_interest_tweens
Age_interest_teens
Age_interest_adults
Height_req_inches
Ride_duration_min
Age_of_ride_days
Age_of_ride_years
TL_rank
TA_Stars
DAYOFWEEK
DAYOFYEAR
WEEKOFYEAR
MONTHOFYEAR
YEAR
HOLIDAYPX


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HOLIDAYM


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HOLIDAY
WDWevent
WDWMAXTEMP
WDWMINTEMP
WDWMEANTEMP
MKevent
EPevent
HSevent
inSession


...

Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HSSHWNGT
HSSHWNT1


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HSSHWNT2
AKPRDDT1
AKPRDDT2
AKSHWNGT
AKSHWNT1
AKSHWNT2
new_case
Wind Angle
Wind Speed
Cloud Height
Visibility Distance (M)
Temperature (C)
Weather Type
WDW_TICKET_SEASON_none
WDW_TICKET_SEASON_peak
WDW_TICKET_SEASON_regular
WDW_TICKET_SEASON_value
SEASON_christmas
SEASON_christmas peak
SEASON_columbus day
SEASON_easter
SEASON_fall
SEASON_halloween
SEASON_jersey week
SEASON_july 4th
SEASON_mardi gras
SEASON_martin luther king junior day
SEASON_memorial day
SEASON_none
SEASON_presidents week
SEASON_september low
SEASON_spring
SEASON_summer break
SEASON_thanksgiving
SEASON_winter
HOLIDAYN_ash
HOLIDAYN_ash|val
HOLIDAYN_cdm
HOLIDAYN_chv
HOLIDAYN_chv|pas
HOLIDAYN_cmd
HOLIDAYN_cmd|han
HOLIDAYN_cme
HOLIDAYN_col
HOLIDAYN_col|suk
HOLIDAYN_elc
HOLIDAYN_esm
HOLIDAYN_ess
HOLIDAYN_fat
HOLIDAYN_gfr
HOLIDAYN_hal
HOLIDAYN_hal|nvd
HOLIDAYN_han
HOLIDAYN_ind
HOLIDAYN_lab
HOLIDAYN_mem
HOLIDAYN_mgs
HOLIDAYN_mlk
HOLIDAYN_mot
HOLIDAYN_njc
HOLIDAYN_njc|vet
HOLIDAYN_none
HOLIDAYN_nvd
HOLIDAYN_nyd
HOLIDAYN_nye
HO

Pipeline(steps=[('imputerAndLogTransformer',
                 FunctionTransformer(func=<function imputeTransform at 0x7f41c4808830>)),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('variancethreshold',
                                                  VarianceThreshold(threshold=0.001),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f41c47fcd50>),
                                                 ('robustscaler',
                                                  RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f41c4818950>)])),
                ('regressor',
                 RandomForestRegressor(max_depth=50, n_estimators=50, n_jobs=-1,
                                       random_state=0))])

In [15]:
pred = clf.predict(X_test_clean)

Ride_type_thrill
Ride_type_spinning
Ride_type_slow
Ride_type_small_drops
Ride_type_big_drops
Ride_type_dark
Fast_pass
Classic
Age_interest_preschoolers
Age_interest_tweens
Age_interest_teens
Age_interest_adults
Height_req_inches
Ride_duration_min
Age_of_ride_days
Age_of_ride_years
TL_rank
TA_Stars
DAYOFWEEK
DAYOFYEAR
WEEKOFYEAR
MONTHOFYEAR
YEAR
HOLIDAYPX


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HOLIDAYM


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HOLIDAY
WDWevent
WDWMAXTEMP
WDWMINTEMP
WDWMEANTEMP
MKevent
EPevent
HSevent
inSession


...

HSFIRET2


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HSSHWNGT
HSSHWNT1


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

HSSHWNT2
AKPRDDT1
AKPRDDT2
AKSHWNGT
AKSHWNT1
AKSHWNT2
new_case
Wind Angle
Wind Speed
Cloud Height
Visibility Distance (M)
Temperature (C)
Weather Type
WDW_TICKET_SEASON_none
WDW_TICKET_SEASON_peak
WDW_TICKET_SEASON_regular
WDW_TICKET_SEASON_value
SEASON_christmas
SEASON_christmas peak
SEASON_columbus day
SEASON_easter
SEASON_fall
SEASON_halloween
SEASON_jersey week
SEASON_july 4th
SEASON_mardi gras
SEASON_martin luther king junior day
SEASON_memorial day
SEASON_none
SEASON_presidents week
SEASON_september low
SEASON_spring
SEASON_summer break
SEASON_thanksgiving
SEASON_winter
HOLIDAYN_ash
HOLIDAYN_ash|val
HOLIDAYN_cdm
HOLIDAYN_chv
HOLIDAYN_chv|pas
HOLIDAYN_cmd
HOLIDAYN_cmd|han
HOLIDAYN_cme
HOLIDAYN_col
HOLIDAYN_col|suk
HOLIDAYN_elc
HOLIDAYN_esm
HOLIDAYN_ess
HOLIDAYN_fat
HOLIDAYN_gfr
HOLIDAYN_hal
HOLIDAYN_hal|nvd
HOLIDAYN_han
HOLIDAYN_ind
HOLIDAYN_lab
HOLIDAYN_mem
HOLIDAYN_mgs
HOLIDAYN_mlk
HOLIDAYN_mot
HOLIDAYN_njc
HOLIDAYN_njc|vet
HOLIDAYN_none
HOLIDAYN_nvd
HOLIDAYN_nyd
HOLIDAYN_nye
HO

In [18]:
mae = metrics.mean_absolute_error(y_test, pred)
mse = metrics.mean_squared_error(y_test, pred)
rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))
r2 = metrics.r2_score(y_test, pred)

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-SQUARED: ", r2)

MAE:  32.50105255564034
MSE:  14652.832199270018
RMSE:  121.04888351104283
R-SQUARED:  0.8267373419589401


In [20]:
print("(PREDICTED, ACTUAL)")
for x in zip(np.round(pred[3000:3020], 0), y_test[3000:3020].values):
    print(x)

(PREDICTED, ACTUAL)
(29.0, 30)
(23.0, 30)
(23.0, 30)
(15.0, 20)
(23.0, 20)
(60.0, 70)
(29.0, 30)
(6.0, 5)
(27.0, 20)
(25.0, 30)
(12.0, 20)
(12.0, 20)
(77.0, 95)
(27.0, 30)
(12.0, 10)
(29.0, 25)
(10.0, 10)
(6.0, 5)
(55.0, 55)
(34.0, 40)
